### Example 1

Suppose that you have 3 machines to manufacture shoes, and the cost of each machine is:
$$𝐶_1 = 0.01𝑛_1^2 + 2𝑛_1$$
$$𝐶_2 = 6𝑛_2$$
$$𝐶_3 = 7𝑛_3$$
where $𝐶_𝑖$ is cost for production of machine $i$, $𝑛_𝑖$ is the number of shoes manufactured in machine $i$.
Each machine has a limit of production of 1.000 shoes. For a total production of 2.100 shoes, how many shoes should each machine made in order to minimize the total cost?


$$\begin{align*} \min \quad & C_1 + C_2 + C_3 \\
\mathrm{s.t.} \quad & 𝐶_1 = 0.01𝑛_1^2 + 2𝑛_1\\
& 𝐶_2 = 6𝑛_2 \\
& 𝐶_3 = 7𝑛_3 \\
& 𝐶_3 = 7𝑛_3 \\
& n_1 + n_2 + n_3 = 2100\\
& 0 \leq n_1, n_2, n_3 \leq 1000
&& n_i \in \mathrm{integers}
\end{align*} $$

In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
m = pe.ConcreteModel()

m.S = pe.RangeSet(1,3)
m.n = pe.Var(m.S, domain=pe.NonNegativeIntegers, bounds=(0,1000))

m.cons1 = pe.Constraint(expr=(pe.summation(m.n) == 2100)) # pe.summation sums all variables from all sets
m.C1 = pe.Expression(expr=0.01*m.n[1]**2 + 2*m.n[1])
m.C2 = pe.Expression(expr=6*m.n[2])
m.C3 = pe.Expression(expr=7*m.n[3])

m.obj = pe.Objective(expr=m.C1 + m.C2 + m.C3,sense=pe.minimize)

In [3]:
solver = po.SolverFactory('couenne') 
results = solver.solve(m)

# If solved with gurobi
# solver = po.SolverFactory('gurobi')
# Problem can be solved mainly due to the fact that it is a SOCP

for s in m.S:
    print(f"x{s}: {pe.value(m.n[s])}")

x1: 250.0
x2: 1000.0
x3: 850.0000000000001


**Now suppose that the cost of the second machine changes to**:
$$ C_2 = 6n_2n_1$$

The problem is not a SOCP anymore it becomes a Nonconvex QP

$$\begin{align*} \min \quad & C_1 + C_2 + C_3 \\
\mathrm{s.t.} \quad & 𝐶_1 = 0.01𝑛_1^2 + 2𝑛_1\\
& 𝐶_2 = 6𝑛_2n_1 \\
& 𝐶_3 = 7𝑛_3 \\
& 𝐶_3 = 7𝑛_3 \\
& n_1 + n_2 + n_3 = 2100\\
& 0 \leq n_1, n_2, n_3 \leq 1000
&& n_i \in \mathrm{integers}
\end{align*} $$

In [4]:
m = pe.ConcreteModel()

m.S = pe.RangeSet(1,3)
m.n = pe.Var(m.S, domain=pe.NonNegativeIntegers, bounds=(0,1000))

m.cons1 = pe.Constraint(expr=(pe.summation(m.n) == 2100)) # pe.summation sums all variables from all sets
m.C1 = pe.Expression(expr=0.01*m.n[1]**2 + 2*m.n[1])
m.C2 = pe.Expression(expr=6*m.n[2]*m.n[1])
m.C3 = pe.Expression(expr=7*m.n[3])

m.obj = pe.Objective(expr=m.C1 + m.C2 + m.C3,sense=pe.minimize)

solver = po.SolverFactory('couenne')
results = solver.solve(m)

# If solved with gurobi
# solver = po.SolverFactory('gurobi')
# solver.options["NonConvex"] = 2

for s in m.S:
    print(f"x{s}: {pe.value(m.n[s])}")

x1: 100.00000000000001
x2: 999.9999999999998
x3: 1000.0
